In [6]:
import pandas as pd 
import numpy as np
import pandas_market_calendars as mcal
from os import listdir
from os.path import isfile, join
import plotly.express as px
from dotmap import DotMap
import yaml
import dask.dataframe as dd


import pytz
eastern = pytz.timezone('US/Eastern')
nyse_cal = mcal.get_calendar('NYSE')


config = DotMap(yaml.safe_load(open("src/config.yaml")), _dynamic=False)

[Python numpy: cannot convert datetime64[ns] to datetime64[D] (to use with Numba)](https://stackoverflow.com/a/76139900/9079015)

In [4]:
def get_next_available_candle(prices: pd.DataFrame, 
                              time: pd.Timestamp) -> pd.Series:
    entry_candle_idx = prices.index.get_indexer(target=[time], 
                                                method="bfill")
    entry_candle = prices.take(entry_candle_idx).iloc[0]
    return entry_candle

In [67]:
def get_appropriate_closing_time(time: pd.Timestamp, tz="US/Eastern") -> pd.Timestamp:
    if (time.hour < 9) or ((time.hour == 9) and (time.minute < 30)):
        return pd.Timestamp(year=time.year, month=time.month, day=time.day, hour=9, minute=31, tz=tz)
    else:
        valid_days = [x.date() for x in nyse_cal.valid_days(start_date=time.date(), end_date=time.date() + pd.DateOffset(days=10))]
        i = 1
        while True:
            new_time = time + pd.DateOffset(days=i)
            if new_time.date() in valid_days:
                return pd.Timestamp(year=new_time.year, month=new_time.month, day=new_time.day, hour=16, minute=1, tz=tz)
            if i == 7:
                return ValueError()
            i += 1

## Import and Preprocess News 

In [68]:
# df = dd.read_parquet(path="data/processed_news")
df = pd.read_parquet(path="data/processed_news/data-0.parquet")
df["time"] = df.time.dt.tz_convert(eastern)

# TODO: This can be *improved* by saying that if we are very close to completing the minute e.g. :55, 
# then we dont take the next candle (T+1), but the candle after the next(T+2).
df["entry_time"] = df["time"].dt.ceil("min")

# Necessary to get `us` units, otherwise pandas will always convert back to `ns` for some reason.
df["nn_exit_time"] = df["time"].map(get_appropriate_closing_time)

In [69]:
df["time"] = df.time.dt.tz_convert(eastern)

# TODO: This can be *improved* by saying that if we are very close to completing the minute e.g. :55, 
# then we dont take the next candle (T+1), but the candle after the next(T+2).
df["entry_time"] = df["time"].dt.ceil("min")

# Necessary to get `us` units, otherwise pandas will always convert back to `ns` for some reason.
df["nn_exit_time"] = df["time"].map(get_appropriate_closing_time)

## Import and Preprocess Stock Prices

In [70]:
# TODO: preprocessing is largely done in iq_feed_cleaning... probably can just import and use as is 
spy: pd.DataFrame = pd.read_parquet(path=f"{config.data.iqfeed.minute.cleaned}/SPY_1min.parquet")
spy.columns = [x.strip("adj_") for x in spy.columns]

In [71]:
onlyfiles = [f for f in listdir(config.data.iqfeed.minute.cleaned) if isfile(join(config.data.iqfeed.minute.cleaned, f))]
tickers = [x.split("_")[0] for x in onlyfiles]
ticker = df.stocks.iloc[0]

In [72]:
prices: pd.DataFrame = pd.read_parquet(path=f"{config.data.iqfeed.minute.cleaned}/{ticker}_1min.parquet")
prices.columns = [x.strip("adj_") for x in prices.columns]
prices["time"] = prices.time.dt.tz_localize(None).dt.tz_localize("US/Eastern")
prices = prices.sort_values("time")

## Handle Ticker:Company Mapping

In [73]:
"""
Handling of multiple tickers for a the same company.
If there is only one price time series available for the company, we simply group together the tickers.
However in some cases we will have multiple price time series for the same company.

E.g. in case of Alphabet (Google) we have two different tickers and two different stock prices for the same
underlying company. Here `GOOG` and `GOOGL` describe two different classes of stock for the same company.
In this case we will try to only look at the main class. 

We find this class by choosing the Symbol with the longer stock price history, assuming that the history
of it includes(!) the history of the other one completely.
If one time series doesn't include the other we merge the two time series. Ideally based on which time series has more liquidity 
in a given week or but we will simply decide that the newer time series takes precedence for simplicity. 
"""

"\nHandling of multiple tickers for a the same company.\nIf there is only one price time series available for the company, we simply group together the tickers.\nHowever in some cases we will have multiple price time series for the same company.\n\nE.g. in case of Alphabet (Google) we have two different tickers and two different stock prices for the same\nunderlying company. Here `GOOG` and `GOOGL` describe two different classes of stock for the same company.\nIn this case we will try to only look at the main class. \n\nWe find this class by choosing the Symbol with the longer stock price history, assuming that the history\nof it includes(!) the history of the other one completely.\nIf one time series doesn't include the other we merge the two time series. Ideally based on which time series has more liquidity \nin a given week or but we will simply decide that the newer time series takes precedence for simplicity. \n"

In [102]:
ticker = "GOOGL"

In [231]:
def get_time_interval(ticker):
    try:
        price : pd.DataFrame = pd.read_parquet(path=f"{config.data.iqfeed.minute.cleaned}/{ticker}_1min.parquet")
    except FileNotFoundError:
        return (np.NaN, np.NaN)
    time_interval = price.index.min(), price.index.max()
    return time_interval

In [232]:
ticker_mapper = pd.read_parquet("data_shared/ticker_name_mapper_reduced.parquet")
ticker_mapper[["first_date", "last_date"]] = np.NaN

In [234]:
for i in ticker_mapper.index:
    ticker_mapper.loc[i, ["first_date", "last_date"]] = get_time_interval(ticker_mapper.loc[i, "stocks"])

In [ ]:
ticker_mapper.dropna(inplace=True)
ticker_mapper[["first_date", "last_date"]] = ticker_mapper[["first_date", "last_date"]].apply(pd.to_datetime, axis=0)

In [220]:
def consolidate_tickers(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(["last_date", "first_date"], ascending=[False, True])
    df.loc[df.index[0], "is_primary_ticker"] = True
    return df.iloc[:1, :]

In [236]:
ticker_mapper.shape

(9670, 5)

In [238]:
ticker_mapper_consolidated = ticker_mapper.copy(deep=True)
ticker_mapper_consolidated["is_primary_ticker"] = np.NaN
ticker_mapper_consolidated = ticker_mapper_consolidated.groupby("company_name").apply(consolidate_tickers)
ticker_mapper_consolidated.shape

In [ ]:
# df = news
def get_primary_ticker(ticker, mapper):
    company_name = mapper.loc[mapper["stocks"] == ticker, "company_name"]
    primary_ticker = mapper.loc[(mapper.company_name == company_name) & mapper.is_primary_ticker, "ticker"]
    return primary_ticker

# Overwrite tickers with consolidated ticker, i.e. the ticker of which we use the time series to construct input-output pairs
df["ticker"] = df.ticker.map(lambda ticker: get_primary_ticker(ticker), mapper=ticker_mapper_consolidated)

## Merge

In [74]:
# TODO: Also merge with  non-adjusted prices. We don't trade penny stocks.
# If the price is smaller than 1 when the news come out we don't trade.

In [75]:
ticker_news = df[df.stocks == ticker]
merged = pd.merge_asof(ticker_news, prices, left_on="entry_time", right_on="time", direction="forward", suffixes=("_news", "_price"))

In [76]:
merged.head()

,time_news,stocks,title,channels,body,author,company_name,short_name,parsed_body,entry_time,nn_exit_time,time_price,open,high,low,close,volume
0,2010-01-03 18:05:02-05:00,JBLU,JetBlue to Waive Change Fees and Fare Differen...,[],"NEW YORK, Jan. 3 /PRNewswire-FirstCall/ -- Jet...",PRNewswire,JetBlue Airways Corporation,JetBlue Airways,FirstCall the company will waive change...,2010-01-03 18:06:00-05:00,2010-01-04 16:01:00-05:00,2010-01-04 08:06:00-05:00,5.55,5.55,5.55,5.55,200.0
1,2010-01-04 11:00:03-05:00,JBLU,Celebrate 2010 with JetBlue Airways' Shake Up ...,[],"NEW YORK, Jan. 4 /PRNewswire-FirstCall/ -- To ...",PRNewswire,JetBlue Airways Corporation,JetBlue Airways,"FirstCall To celebrate 2010, the company ...",2010-01-04 11:01:00-05:00,2010-01-05 16:01:00-05:00,2010-01-04 11:01:00-05:00,5.57,5.58,5.56,5.58,5100.0
2,2010-01-05 09:03:43-05:00,JBLU,JetBlue Expands in San Francisco Today,[],"SAN FRANCISCO, Jan. 5 /PRNewswire-FirstCall/ -...",PRNewswire,JetBlue Airways Corporation,JetBlue Airways,FirstCall the company today kicks off a...,2010-01-05 09:04:00-05:00,2010-01-05 09:31:00-05:00,2010-01-05 09:04:00-05:00,5.50,5.50,5.50,5.50,525.0


In [77]:
merged = pd.merge(merged, prices, left_on="nn_exit_time", right_on="time", suffixes=("_entry", "_exit"))
merged["r"] = merged["open_exit"] / merged["open_entry"] - 1

In [ ]:
# Ideally we do this for every stock first and then we come back with the complete dataframe... (depends on if it fits in memory)
# Merge news and stock prices with spy prices
merged = pd.merge_asof(merged, spy, left_on="entry_time", right_on="time", direction="forward")

# TODO: Don't use intraday as exit here (closing candle) but the actual closing auction...
# But for that we need the daily time series, not with minute frequency
merged = pd.merge_asof(merged, spy, left_on="nn_exit_time", right_on="time", suffixes=("_spy_entry", "_spy_exit"))

merged.loc[:, "r_spy"] = merged["close_spy_exit"] / merged["close_spy_entry"] - 1
merged.loc[:, "r_mkt_adj"] = merged["r_spy"] - merged["r"]

In [ ]:
merged.head(3)

In [ ]:
merged = merged.loc[:, ["time", "stocks", "body", "entry_time", "r_mkt_adj"]]

In [ ]:
merged.dtypes

In [ ]:
merged.head(3)

# Save to Files 

Columns: `time|  stocks  |body  |entry_time  |r_mkt_adj`

In [ ]:
# Splitting training and test set
merged_test = merged.loc[merged.time >= config.model.data.cutoff_date]
merged_train = merged.loc[merged.time < config.model.data.cutoff_date]
assert merged_test.shape[0] + merged_train.shape[0] == merged.shape[0]

In [ ]:
merged_train

In [ ]:
config.model.data.training
config.model.data.testing

In [ ]:
prices[(prices.index.hour == 16) & (prices.index.minute==0)]